# Loss Ratio Prediction

In [56]:
import os
import math
import csv
import collections
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline

## Load Dataset

In [2]:
path_training_dataset = "../../dataset/training_data.csv"
df_data = pd.read_csv(path_training_dataset)

c:\users\adity\venv\data-science\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_data.head()

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0


In [4]:
df_data.columns

Index(['PolicyNo', 'Policy_Company', 'Policy_Installment_Term',
       'Policy_Billing_Code', 'Policy_Method_Of_Payment',
       'Policy_Reinstatement_Fee_Indicator',
       'Policy_Zip_Code_Garaging_Location', 'Vehicle_Territory',
       'Vehicle_Make_Year', 'Vehicle_Make_Description', 'Vehicle_Performance',
       'Vehicle_New_Cost_Amount', 'Vehicle_Symbol',
       'Vehicle_Number_Of_Drivers_Assigned', 'Vehicle_Usage',
       'Vehicle_Miles_To_Work', 'Vehicle_Days_Per_Week_Driven',
       'Vehicle_Annual_Miles', 'Vehicle_Anti_Theft_Device',
       'Vehicle_Passive_Restraint', 'Vehicle_Age_In_Years',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Bodily_Injury_Limit',
       'Vehicle_Physical_Damage_Limit',
       'Vehicle_Comprehensive_Coverage_Indicator',
       'Vehicle_Comprehensive_Coverage_Limit',
       'Vehicle_Collision_Coverage_Indicator',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Total',
       'Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single',
     

### Common Functions

In [5]:
def get_selected_features(feature_list_file_path):
    with open(feature_list_file_path) as f:
        features_list = f.read().splitlines() 
        return features_list

In [6]:
def strip_white_spaces(input_df):
    columns = list(input_df.columns)
    for column in columns:
        input_df[column] = input_df[column].str.strip()
    return input_df

In [7]:
def convert_df_objects_to_str(df_dataset, df_columns):
    for column in df_columns:
        if df_dataset[column].dtype == 'object':
            df_dataset[column] = df_dataset[column].astype(str)
    return df_dataset

In [8]:
def get_num_cat_features(df_dataset, df_columns):
    categorical_variables = []
    numerical_variables = []
    for column in df_columns:
        print(column)
        if df_dataset[column].dtype == 'object':
            categorical_variables.append(column)
        else:
            numerical_variables.append(column)
    return numerical_variables, categorical_variables

In [9]:
selected_features_train = [
    'Vehicle_Youthful_Good_Student_Code',
    'Vehicle_Driver_Points',
    'Driver_Total_Female',
    'Driver_Total_Male',
    'Driver_Total_Teenager_Age_15_19',
    'Driver_Total_College_Ages_20_23',
    'Driver_Total_Young_Adult_Ages_24_29',
    'Driver_Total_Low_Middle_Adult_Ages_30_39',
    'Driver_Total_Middle_Adult_Ages_40_49',
    'Driver_Total_Adult_Ages_50_64',
    'Driver_Total_Senior_Ages_65_69',
    'Driver_Total_Upper_Senior_Ages_70_plus',
    'Driver_Total_Married',
    'Driver_Total_Single',
    'Vehicle_Usage',
    'Vehicle_Miles_To_Work',
    'Vehicle_Territory',
    'Annual_Premium',
    'Loss_Amount']

In [10]:
df_data = df_data[selected_features_train]

In [11]:
num_features, cat_features = get_num_cat_features(df_data, list(df_data.columns))

Vehicle_Youthful_Good_Student_Code
Vehicle_Driver_Points
Driver_Total_Female
Driver_Total_Male
Driver_Total_Teenager_Age_15_19
Driver_Total_College_Ages_20_23
Driver_Total_Young_Adult_Ages_24_29
Driver_Total_Low_Middle_Adult_Ages_30_39
Driver_Total_Middle_Adult_Ages_40_49
Driver_Total_Adult_Ages_50_64
Driver_Total_Senior_Ages_65_69
Driver_Total_Upper_Senior_Ages_70_plus
Driver_Total_Married
Driver_Total_Single
Vehicle_Usage
Vehicle_Miles_To_Work
Vehicle_Territory
Annual_Premium
Loss_Amount


In [12]:
num_features

['Vehicle_Driver_Points',
 'Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount']

In [13]:
cat_features

['Vehicle_Youthful_Good_Student_Code', 'Vehicle_Usage']

### Cleaning Training Dataset

In [14]:
df_num = df_data[num_features]
df_num.head()

,Vehicle_Driver_Points,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount
0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0
1,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0
2,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0
3,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0
4,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0


In [15]:
df_cat = df_data[cat_features]
df_cat.head()

,Vehicle_Youthful_Good_Student_Code,Vehicle_Usage
0,Not Eligible for Good Student Credit,Farm
1,Not Eligible for Good Student Credit,Farm
2,Not Eligible for Good Student Credit,Pleasure
3,Not Eligible for Good Student Credit,Pleasure
4,Not Eligible for Good Student Credit,Pleasure


In [16]:
df_cat_strip = strip_white_spaces(df_cat)

c:\users\adity\venv\data-science\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
df_data_clean = df_num.join(df_cat, how='outer')

In [18]:
df_data_clean.head()

,Vehicle_Driver_Points,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount,Vehicle_Youthful_Good_Student_Code,Vehicle_Usage
0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0,Not Eligible for Good Student Credit,Farm
1,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0,Not Eligible for Good Student Credit,Farm
2,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0,Not Eligible for Good Student Credit,Pleasure
3,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Not Eligible for Good Student Credit,Pleasure
4,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Not Eligible for Good Student Credit,Pleasure


### Find useful features

In [19]:
def get_unique_values_with_count(df_input):
    columns = df_input.columns
    for column in columns:
        unique_value_count = df_input[column].value_counts()
        print(column)
        print()
        print(unique_value_count)
        print('--'*50)

In [20]:
get_unique_values_with_count(df_data)

Vehicle_Youthful_Good_Student_Code

Not Eligible for Good Student Credit    418562
Eligible for Good Student Credit          5869
Name: Vehicle_Youthful_Good_Student_Code, dtype: int64
----------------------------------------------------------------------------------------------------
Vehicle_Driver_Points

0    381289
1     36431
2      5905
3       670
4       108
5        21
6         5
7         2
Name: Vehicle_Driver_Points, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Female

1    238676
0    181914
2      3837
3         4
Name: Driver_Total_Female, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Male

1    222967
0    198925
2      2537
3         2
Name: Driver_Total_Male, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Teenager_Age_15_19

0  

### Features to be used in portfolio

- Vehicle_Make_Year_Mean
- Vehicle_Performance_Standard_Minus_all
- Vehicle_Number_Of_Drivers_Assigned_Mean_Ignore_gt_10
- Vehicle_Usage_Pleasure_Minus_all
- Vehicle_Anti_Theft_Device_anything_but_not_applicable
- Vehicle_Age_In_Years_Mean
- Vehicle_Collision_Coverage_Indicator_Y_minus_N
- Driver_Total_Single_Sum_not_include_0
- Driver_Total_Mean
- Driver_Total_Male_Mean
- Driver_Total_Female_Mean
- Driver_Total_Married_Mean
- Driver_Total_Teenager_Age_15_19_Mean
- Driver_Total_College_Ages_20_23_Sum
- Driver_Total_Young_Adult_Ages_24_29_Mean
- Driver_Total_Upper_Senior_Ages_70_plus_Mean
- Annual_Premium_Sum

In [21]:
selected_features_test = ['Vehicle_Make_Year',
                        'Vehicle_Number_Of_Drivers_Assigned',
                        'Vehicle_Age_In_Years',
                        'Driver_Total_Single',
                        'Driver_Total',
                        'Driver_Total_Male',
                        'Driver_Total_Female',
                        'Driver_Total_Married',
                        'Driver_Total_Teenager_Age_15_19',
                        'Driver_Total_College_Ages_20_23',
                        'Driver_Total_Young_Adult_Ages_24_29',
                        'Driver_Total_Upper_Senior_Ages_70_plus',
                        'Vehicle_Usage',
                        'Vehicle_Anti_Theft_Device',
                        'Vehicle_Performance',
                        'Annual_Premium']

### Create Portfolios in training data like testing portfolios

In [22]:
df_data.shape

(424431, 19)

In [23]:
policies_in_portfolios = [1000, 3000, 5000]
percentage_losses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

In [24]:
df_list_portfolios = []

In [25]:
df_data_oh = pd.get_dummies(df_data)

In [26]:
df_data_oh.shape

(424431, 23)

In [27]:
df_data_oh_columns = list(df_data_oh.columns)

In [28]:
for i in range(10):
    df_data = df_data_oh.sample(frac=1)
    df_data_loss_zero = df_data[df_data['Loss_Amount'] == 0]
    df_data_loss_non_zero = df_data[df_data['Loss_Amount'] != 0]
    current_df_without_loss = 0
    current_df_with_loss = 0
    for i in range(len(policies_in_portfolios)):
        for j in range(len(percentage_losses)):
            without_loss_size = int(((100 - percentage_losses[j])/100) * policies_in_portfolios[i])
            with_loss_size = int(policies_in_portfolios[i] - without_loss_size )

            df_without_loss = df_data_loss_zero.iloc[current_df_without_loss: current_df_without_loss + without_loss_size]
            df_with_loss = df_data_loss_non_zero.iloc[current_df_with_loss: current_df_with_loss + with_loss_size]

            current_df_without_loss = current_df_without_loss + without_loss_size
            current_df_with_loss = current_df_with_loss + with_loss_size

            df_merge = pd.concat([df_without_loss, df_with_loss])
            df_list_portfolios.append(df_merge)

In [29]:
len(df_list_portfolios)

330

In [30]:
df_list_portfolios[0].shape

(1000, 23)

In [31]:
df_data_final = None

In [32]:
df_data_oh_columns

['Vehicle_Driver_Points',
 'Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [33]:
selected_features_mean = num_features.copy()
selected_features_mean.remove('Annual_Premium')
selected_features_mean.remove('Loss_Amount')
mean_columns = selected_features_mean

sum_columns = ['Annual_Premium']

for cat_f in cat_features:
    for col in list(df_list_portfolios[0].columns):
        if col.startswith(cat_f):
            sum_columns.append(col)

target_column = ['Loss_Amount']

In [34]:
sum_columns

['Annual_Premium',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [35]:
ls_final_training = []

In [36]:
def portfolio_to_features(df_portfolio, is_training=True):
    df_portfolio_mean = df_portfolio[mean_columns]
    df_portfolio_mean = df_portfolio_mean.mean()
    ls_portfolio_mean = df_portfolio_mean.to_list()
    
    df_portfolio_sum = df_portfolio[sum_columns]
    df_portfolio_sum = df_portfolio_sum.sum()
    ls_portfolio_sum = df_portfolio_sum.to_list()
    
    if is_training:
        df_portfolio_target = df_portfolio[target_column]
        df_portfolio_target_sum = df_portfolio_target.sum()
        ls_portfolio_target_sum = df_portfolio_target_sum.to_list()
    
    list_merge = ls_portfolio_mean + ls_portfolio_sum
    
    if is_training:
        list_merge = list_merge + ls_portfolio_target_sum
    
    return list_merge

In [37]:
def features_to_training_df(df_portfolios_list):
    features_list = mean_columns + sum_columns + target_column
    final_list = []
    for single_portfolio in df_portfolios_list:
        ls_portfolio = portfolio_to_features(single_portfolio)
        final_list.append(ls_portfolio)
    df_final_training = pd.DataFrame(final_list, columns = features_list)
    return df_final_training

In [38]:
df_final_train = features_to_training_df(df_list_portfolios)

In [39]:
df_final_train.head()

,Vehicle_Driver_Points,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,...,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit,Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit,Vehicle_Usage_Business,Vehicle_Usage_Farm,Vehicle_Usage_Pleasure,Vehicle_Usage_Work,Loss_Amount
0,0.121,0.616,0.495,0.029,0.036,0.083,0.227,0.276,0.316,0.058,...,3.823,30.945,343550.39,13.0,987.0,11.0,99.0,520.0,370.0,19846.99
1,0.106,0.594,0.532,0.042,0.044,0.077,0.218,0.288,0.297,0.056,...,3.106,31.218,333257.72,14.0,986.0,0.0,88.0,526.0,386.0,60491.98
2,0.112,0.593,0.528,0.055,0.050,0.085,0.223,0.262,0.277,0.060,...,3.312,31.015,344109.92,17.0,983.0,6.0,108.0,529.0,357.0,145104.94
3,0.101,0.541,0.559,0.034,0.038,0.086,0.202,0.263,0.313,0.069,...,2.761,31.112,336219.83,11.0,989.0,3.0,95.0,575.0,327.0,292311.05
4,0.106,0.566,0.529,0.036,0.037,0.072,0.207,0.264,0.318,0.067,...,3.611,30.919,327560.76,12.0,988.0,11.0,98.0,526.0,365.0,196534.94


### Training the model with Decision Tree

In [40]:
X_train = df_final_train.loc[:, df_final_train.columns != 'Loss_Amount']
y_train = df_final_train['Loss_Amount']

In [41]:
X_train.shape, y_train.shape

((330, 22), (330,))

In [42]:
X_train.columns

Index(['Vehicle_Driver_Points', 'Driver_Total_Female', 'Driver_Total_Male',
       'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Driver_Total_Married',
       'Driver_Total_Single', 'Vehicle_Miles_To_Work', 'Vehicle_Territory',
       'Annual_Premium',
       'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
       'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
       'Vehicle_Usage_Business', 'Vehicle_Usage_Farm    ',
       'Vehicle_Usage_Pleasure', 'Vehicle_Usage_Work    '],
      dtype='object')

In [43]:
decision_tree_regressor = DecisionTreeRegressor()
decision_tree_regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [44]:
y_train_pred = decision_tree_regressor.predict(X_train)

In [45]:
mae = mean_absolute_error(y_train, y_train_pred)
mae

0.0

In [46]:
from xgboost import XGBRegressor
xg_regressor = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 3, alpha = 10, n_estimators = 100)
xg_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

c:\users\adity\venv\data-science\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\adity\venv\data-science\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [47]:
y_train_pred_xg = xg_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [48]:
mae_xg = mean_absolute_error(y_train, y_train_pred_xg)
mae_xg

526990.2544564393

## Train Using KNN Model

In [57]:
knn_regressor = KNeighborsRegressor(5)
knn_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [58]:
y_train_pred_knn = knn_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [59]:
mae_knn = mean_absolute_error(y_train, y_train_pred_knn)
mae_knn

260042.78166060607

In [49]:
def load_test_portfolios():
    path_testing_dataset = "../dataset/testing_portfolios"
    portfolio_files = os.listdir(path_testing_dataset)
    portfolio_dict = {}
    for i in range(len(portfolio_files)):
        portfolio_id = int(portfolio_files[i].split('.')[0][15:])
        portfolio_dict[portfolio_id] = portfolio_files[i]
    od = collections.OrderedDict(sorted(portfolio_dict.items()))
    po_list = []
    for key,val in od.items():
        po_list.append(val)
    df_portfolio_list = []
    for portfolio_file in po_list:
        file_path = os.path.join(path_testing_dataset, portfolio_file)
        df_test_portfolio = pd.read_csv(file_path)
        test_num_features = num_features.copy()
        test_num_features.remove('Loss_Amount')
        
        df_test_num = df_test_portfolio[test_num_features]
        df_test_cat = df_test_portfolio[cat_features]
        df_test_cat_strip = strip_white_spaces(df_test_cat)
        df_test_portfolio = df_test_num.join(df_test_cat_strip, how='outer')
        df_test_portfolio = pd.get_dummies(df_test_portfolio)
        df_test_portfolio = df_test_portfolio.reindex(sorted(X_train.columns), axis=1)
        
        df_portfolio_list.append(df_test_portfolio)
        
    result_dict = collections.OrderedDict()
    for i in range(len(po_list)):
        portfolio_id = po_list[i].split('.')[0][5:]
        result_dict[portfolio_id] = df_portfolio_list[i] 
    return result_dict

In [50]:
def features_to_testing_df(df_portfolios_ordered_dict):
    features_list = mean_columns + sum_columns
    features_list.insert(0, 'ID')
    final_list = []
    for portfolio_id, single_portfolio in df_portfolios_ordered_dict.items():
        ls_portfolio = portfolio_to_features(single_portfolio, False)
        ls_portfolio.insert(0, portfolio_id)
        final_list.append(ls_portfolio)
    df_final_testing = pd.DataFrame(final_list, columns = features_list)
    return df_final_testing

In [51]:
test_portfolios_dict = load_test_portfolios()

c:\users\adity\venv\data-science\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [52]:
for key, value in test_portfolios_dict.items():
    print(key, value.shape)

portfolio_1 (1000, 22)
portfolio_2 (999, 22)
portfolio_3 (1000, 22)
portfolio_4 (997, 22)
portfolio_5 (998, 22)
portfolio_6 (1000, 22)
portfolio_7 (1000, 22)
portfolio_8 (998, 22)
portfolio_9 (998, 22)
portfolio_10 (998, 22)
portfolio_11 (1000, 22)
portfolio_12 (999, 22)
portfolio_13 (999, 22)
portfolio_14 (997, 22)
portfolio_15 (1000, 22)
portfolio_16 (999, 22)
portfolio_17 (998, 22)
portfolio_18 (999, 22)
portfolio_19 (998, 22)
portfolio_20 (997, 22)
portfolio_21 (1000, 22)
portfolio_22 (999, 22)
portfolio_23 (1000, 22)
portfolio_24 (1000, 22)
portfolio_25 (998, 22)
portfolio_26 (1000, 22)
portfolio_27 (998, 22)
portfolio_28 (999, 22)
portfolio_29 (998, 22)
portfolio_30 (998, 22)
portfolio_41 (1000, 22)
portfolio_42 (1000, 22)
portfolio_43 (998, 22)
portfolio_44 (999, 22)
portfolio_45 (999, 22)
portfolio_46 (999, 22)
portfolio_47 (998, 22)
portfolio_48 (1000, 22)
portfolio_49 (999, 22)
portfolio_50 (999, 22)
portfolio_51 (999, 22)
portfolio_52 (998, 22)
portfolio_53 (996, 22)
portfol

In [53]:
df_test_final = features_to_testing_df(test_portfolios_dict)

In [54]:
def generate_test_results(regressor, X_test, output_dir):
    # for each policy
    result_data = collections.OrderedDict()
    portfolio_ids_df = X_test.iloc[:, 0:1]
    X_test_features = X_test.iloc[:, 1:]
    
    y_pred = regressor.predict(np.nan_to_num(X_test_features.to_numpy()))
    loss_ratio = y_pred/X_test['Annual_Premium']
    loss_ratio_log = np.log(loss_ratio)
    loss_ratio_log_df = loss_ratio_log.to_frame(name='ln_LR')
    result_df = portfolio_ids_df.join(loss_ratio_log_df, how='outer')
    
    result_df.to_csv('output.csv', index=False)

In [60]:
generate_test_results(knn_regressor, df_test_final, '')